Here the goal is to understand how we can derive the softmax function from first principles. It's a very simple exercise that will show why the softmax makes sense.

Softmax is used in multi class probabilities where a single training example can have multiple classes. Let's say we have 3 types of images with dog, cat and squirrel. There are 3 output classes here instead of the typical binary classification (2 classes) we dealt it. In typical binary classification we relied on the sigmoid to compute the probability for class 0 and then subtract 1 by itself to get the opposing probability. Here however, it isn't simple as there can be n classes. 

The way to handle n classes also stems from a core idea of assigning a single probability to each output class. Of course, all these probabilities will add up to 1 and this is the key for softmax.

Suppose we have an array $A$ and $A$ is the output we predict with exactly $n$ classes. Here it is obvious that whatever the model predicts, the sum of $A$ will not be 1. The goal and design of softmax is to force sum of $A$ to be 1 and make $A$ be the probabilities of each class.

How can we take an array $A$ and force the sum to be 1. One idea is to use ratios so let's try that. We define $f(A)=A/sum(A)$. Here we rank each contribution of an element of $A$ in the total value of $A$, so when elementwise divide, we get the ratios and of course, the sum of $A$ will be one. In equation, $f(A)_i = \frac{A_i}{\sum_{i=1}^n{A_i}}$, where $i$ refers to each element of the array $A$.

Of course, there are problems with this approach. The one glaring problem is negative values. This equation assumes that $A$ will only have positive values. This is not the case as we can get negative inputs for $A$. Then we would be dealing with negative probabilities which makes no sense. What we need is a way to remap the negatives to positive number line while preserving the scales between each value of $A$. For instance, one thought might be to use $|A|$ as a way to force positives but this is wrong since there might be $5$ and $-5$ in $A$, and obviously $|A|$ would rank both of them as being the same. Another way to describe these requirements is to say we need a monotonically increasing function that is always positive over the full domain of values. 

![Monotonic Functions From Wikipedia](https://upload.wikimedia.org/wikipedia/commons/thumb/6/6a/Growth_equations.png/1024px-Growth_equations.png)

These are some example functions from wikipedia that are monotonically increasing functions. Remember, we also need these functions to be strictly positive. From the images above, the functions that can do this are the exponential function and the logistic function. The rest are non increasing or can be negative over the domain. From this, we choose the simplest function which is the exponential function to use as our mapping from $A$. So now, the function becomes $f(A)_i = \frac{e^{A_i}}{\sum_{i=1}^n{e^{A_i}}}$. And voila, here is the softmax function we derived from first principles. Remember, $A$ is a vector and $f(A)$ is also a vector so we have a vector function. This new vector $f(A)$ is now guaranteed to be probabilities that add up to 1. Let's now show the derivative of the softmax with sympy.

In [82]:
import sympy as sp

In [83]:
i = sp.symbols('i', integer=True)
A = sp.MatrixSymbol('A', 3, 1)
softmax_sum = sp.summation(sp.exp(A[i,0]),(i,0,2)) # take full sum of array
softmax_A0 = sp.exp(A[0,0]) / softmax_sum
softmax_A1 = sp.exp(A[1,0]) / softmax_sum
softmax_A2 = sp.exp(A[2,0]) / softmax_sum

softmax_array = [softmax_A0, softmax_A1, softmax_A2]
F = sp.Matrix(softmax_array)
display(softmax_sum)
display(F)

exp(A[0, 0]) + exp(A[1, 0]) + exp(A[2, 0])

Matrix([
[exp(A[0, 0])/(exp(A[0, 0]) + exp(A[1, 0]) + exp(A[2, 0]))],
[exp(A[1, 0])/(exp(A[0, 0]) + exp(A[1, 0]) + exp(A[2, 0]))],
[exp(A[2, 0])/(exp(A[0, 0]) + exp(A[1, 0]) + exp(A[2, 0]))]])

To simplify, i'm only showing an array with 3 elements.

In [84]:
derivatives = sp.derive_by_array(softmax_array, [A[i,0] for i in range(3)])
derivatives = derivatives.simplify()
display(derivatives)
print(sp.latex(derivatives))

[[(exp(A[1, 0]) + exp(A[2, 0]))*exp(A[0, 0])/(exp(A[0, 0]) + exp(A[1, 0]) + exp(A[2, 0]))**2, -exp(A[0, 0] + A[1, 0])/(exp(A[0, 0]) + exp(A[1, 0]) + exp(A[2, 0]))**2, -exp(A[0, 0] + A[2, 0])/(exp(A[0, 0]) + exp(A[1, 0]) + exp(A[2, 0]))**2], [-exp(A[0, 0] + A[1, 0])/(exp(A[0, 0]) + exp(A[1, 0]) + exp(A[2, 0]))**2, (exp(A[0, 0]) + exp(A[2, 0]))*exp(A[1, 0])/(exp(A[0, 0]) + exp(A[1, 0]) + exp(A[2, 0]))**2, -exp(A[1, 0] + A[2, 0])/(exp(A[0, 0]) + exp(A[1, 0]) + exp(A[2, 0]))**2], [-exp(A[0, 0] + A[2, 0])/(exp(A[0, 0]) + exp(A[1, 0]) + exp(A[2, 0]))**2, -exp(A[1, 0] + A[2, 0])/(exp(A[0, 0]) + exp(A[1, 0]) + exp(A[2, 0]))**2, (exp(A[0, 0]) + exp(A[1, 0]))*exp(A[2, 0])/(exp(A[0, 0]) + exp(A[1, 0]) + exp(A[2, 0]))**2]]

\left[\begin{matrix}\frac{\left(e^{A_{1, 0}} + e^{A_{2, 0}}\right) e^{A_{0, 0}}}{\left(e^{A_{0, 0}} + e^{A_{1, 0}} + e^{A_{2, 0}}\right)^{2}} & - \frac{e^{A_{0, 0} + A_{1, 0}}}{\left(e^{A_{0, 0}} + e^{A_{1, 0}} + e^{A_{2, 0}}\right)^{2}} & - \frac{e^{A_{0, 0} + A_{2, 0}}}{\left(e^{A_{0, 0}} + e^{A_{1, 0}} + e^{A_{2, 0}}\right)^{2}}\\- \frac{e^{A_{0, 0} + A_{1, 0}}}{\left(e^{A_{0, 0}} + e^{A_{1, 0}} + e^{A_{2, 0}}\right)^{2}} & \frac{\left(e^{A_{0, 0}} + e^{A_{2, 0}}\right) e^{A_{1, 0}}}{\left(e^{A_{0, 0}} + e^{A_{1, 0}} + e^{A_{2, 0}}\right)^{2}} & - \frac{e^{A_{1, 0} + A_{2, 0}}}{\left(e^{A_{0, 0}} + e^{A_{1, 0}} + e^{A_{2, 0}}\right)^{2}}\\- \frac{e^{A_{0, 0} + A_{2, 0}}}{\left(e^{A_{0, 0}} + e^{A_{1, 0}} + e^{A_{2, 0}}\right)^{2}} & - \frac{e^{A_{1, 0} + A_{2, 0}}}{\left(e^{A_{0, 0}} + e^{A_{1, 0}} + e^{A_{2, 0}}\right)^{2}} & \frac{\left(e^{A_{0, 0}} + e^{A_{1, 0}}\right) e^{A_{2, 0}}}{\left(e^{A_{0, 0}} + e^{A_{1, 0}} + e^{A_{2, 0}}\right)^{2}}\end{matrix}\right]


This is the derivative of the softmax. The rows dictate the 3 different derivatives and the cols dictate the 3 different functions. It looks very patterned but also very cluttered. This is definitely short to implement in actual code however. 